In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv')
df.head()

,Unnamed: 0,Model,Platform,Carrier,Color,Storage,ListDate,ExpiredDate,SaleDate,Views,Quantity,Price,Condition,Description,Damage,Sold
0,LUIW14380,Apple iPhone Xr,iOS,Unlocked,Blue,128 GB,"Aug 18, 2020",NaN,NaN,80,1,529,Mint,\nLike New Device» Mint Condition» Generic Box...,NaN,NaN
